In [ ]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

df_0 = pd.read_excel(r'C:\Users\Riddhima\Downloads\marketing_campaign.xlsx')
df=df_0.copy()
#df.head()
df.info()



In [ ]:
df.describe()
df['customer_lifetime'] = 2024 - df['Dt_Customer'].dt.year
df.head()

In [ ]:
df=df.drop(['Dt_Customer'],axis=1)  # we drop the column 
df.head()


In [ ]:
print((df.isnull().sum()/len(df))*100) # to check the missing values 




In [ ]:
from sklearn.impute import SimpleImputer  #handling the missing values using   Median for the income column 
imputer = SimpleImputer(strategy='median')
df['Income'] = imputer.fit_transform(df[['Income']])

In [ ]:

# Extracting year, month, and day  from Dt_Customer
df['age']=2024 - df["Year_Birth"] #create the age column 


df=df.drop(['Year_Birth'],axis=1)  # we drop the column 


df.info()

In [ ]:
#univariate analysis 
numerical = df.select_dtypes(include=np.number)
for numerical_column in list(numerical):
    plt.figure(figsize=(8, 6))
    sns.displot(df[numerical_column], bins=20, color='blue', alpha=0.7)
    plt.title(f'Distribution of {numerical_column}')
    plt.xlabel(numerical_column)
    plt.ylabel('Frequency')
    plt.grid(True)
    plt.show()

In [ ]:
df.var()




In [ ]:
df.drop('Z_CostContact',axis=1,inplace=True)
df.drop('Z_Revenue',axis=1,inplace=True)  #low  variance in distribution 



In [ ]:
# Bar plot for a categorical variable
catergorical=df.select_dtypes(include=np.object)

for catergorical_col in list(catergorical):
    plt.figure(figsize=(8, 6))
    plt.hist(df[catergorical_col], bins=20, color='blue', alpha=0.7)
    plt.title(f'Distribution of {catergorical_col}')
    plt.xlabel(catergorical_col)
    plt.ylabel('Frequency')
    plt.grid(True)
    plt.show()


In [ ]:
df["Marital_Status"] = df["Marital_Status"].replace({"Married": "Married", "Together": "Married", "Absurd": "Single",'YOLO':"Single",'Alone':"Single",'single':"Single"})
print(set(df['Marital_Status']))

In [ ]:
correlation_matrix = df.corr()
#print(correlation_matrix)
plt.figure(figsize=(20,20))  # Adjust the width and height as needed
sns.heatmap(correlation_matrix, annot=True,  center = 0,cmap='viridis', fmt=".2f")
plt.xticks(rotation=45)
plt.yticks(rotation=0)

plt.title('Correlation Matrix')
plt.show()

In [ ]:
# feature engineering 
from sklearn.preprocessing import LabelEncoder
education_mapping = {        # there is a meaningful ranking 
    'Basic':1,
    '2n Cycle':2,
    'Graduation':3,
    'Master':4,
    'PhD':5
}
print(set(df['Education']))


# Replace the categories with numerical values
df['Education'] = df['Education'].map(education_mapping)

# Perform one-hot encoding
one_hot_encoded = pd.get_dummies(df['Marital_Status'])

# Concatenate the one-hot encoded columns with the original DataFrame
df = pd.concat([df, one_hot_encoded], axis=1)

# Optionally, drop the original categorical column if desired
df.head()


In [ ]:
df.drop('Marital_Status',axis=1,inplace=True)



In [ ]:
import seaborn as sns
 
 # to find outliers 
numerical = df.select_dtypes(include=np.number)

for numerical_column in list(numerical):
    plt.figure(figsize=(8, 6))
    
    # Boxplot
    sns.boxplot(data=df[numerical_column], color='blue')
    
    # Set title and labels
    plt.title(f'Boxplot of {numerical_column}')
    plt.xlabel(numerical_column)
    plt.ylabel('Value')
    
    # Show plot
    plt.show()


In [ ]:
rows_to_delete = df[df['age'] > 100].index

df.drop(rows_to_delete, inplace=True)  # outliers handling delete the id with year of birth less than 1900


In [ ]:
import seaborn as sns
 
 # to find the distribtuion of the data  
numerical = df.select_dtypes(include=np.number)

for numerical_column in list(numerical):
    plt.figure(figsize=(8, 6))
    
    # Boxplot
    sns.displot(data=df[numerical_column], color='blue')
    
    # Set title and labels
    plt.title(f'distribution of {numerical_column}')
    plt.xlabel(numerical_column)
    plt.ylabel('Value')
    
    # Show plot
    plt.show()

In [ ]:
df1=df.copy()


In [ ]:
df.describe()
from sklearn.preprocessing import MinMaxScaler

# Assuming 'data' is your DataFrame
data_to_transform = df[['Income']]  # Select the columns with high std 
scaler = MinMaxScaler()
scaler_fit = scaler.fit(data_to_transform)
normalized_data = scaler_fit.transform(data_to_transform)
df[['Income']]=normalized_data

df.describe()

In [ ]:
'''
Use transformations when features have skewed distributions, 
and you want to make them more suitable for modeling.
(Logarithmic Transformation):used to handle the right skewness

df['MntWines'] = np.log1p(df['MntWines']) 
df['MntFruits'] = np.log1p(df['MntFruits'])
df['MntMeatProducts'] = np.log1p(df['MntMeatProducts'])
df['MntFishProducts'] = np.log1p(df['MntFishProducts'])
df['MntSweetProducts'] = np.log1p(df['MntSweetProducts'])
df['MntGoldProds'] = np.log1p(df['MntGoldProds'])


'''

In [ ]:
df.head()

In [ ]:
# Creating a copy of data
from sklearn.preprocessing import MinMaxScaler,StandardScaler
df_copy = df.copy()

#cols_del = ['AcceptedCmp3', 'AcceptedCmp4', 'AcceptedCmp2','AcceptedCmp5', 'AcceptedCmp1']
#df_copy = df_copy.drop(cols_del, axis=1)
# Scaling all the features inorder to apply the clustering algorithms 
scaler = StandardScaler()
scaler.fit(df_copy)
scaled_ds = pd.DataFrame(scaler.transform(df_copy), columns=df_copy.columns)
scaled_ds.head()

In [ ]:
from pandas import read_csv
import numpy as np
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
#dimensionalty reduction to reduce the no of feature but capture the same amount of variance 
from sklearn.decomposition import PCA
pca = PCA(n_components=3)
pca.fit(scaled_ds)
pca_ds = pd.DataFrame(pca.transform(scaled_ds), columns=(["PC1", "PC2","PC3"]))
pca_ds.describe().T


In [ ]:
# A 3D Projection Of Data In The Reduced Dimension
x = pca_ds["PC1"]
y = pca_ds["PC2"]
z = pca_ds["PC3"]
# Plotting
fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection="3d")
ax.scatter(x, y, z, c="blue", marker="o")
ax.set_title("A 3D Projection Of Data In The Reduced Dimension")
plt.show()

In [ ]:
'''
from sklearn.cluster import KMeans

from yellowbrick.cluster import KElbowVisualizer

# Quick examination of elbow method to find numbers of clusters to make.
print('Elbow Method to determine the number of clusters to be formed:')
visualizer = KElbowVisualizer(KMeans(), k=10,metric='distortion')
visualizer.fit(pca_ds)
visualizer.show()
'''

In [ ]:
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt

# the bend in the elbowcurve is at 4 clusters so n_clusters should be 4
from sklearn.metrics import silhouette_score, davies_bouldin_score, calinski_harabasz_score

kmeans = KMeans(n_clusters=4, random_state=42)

# Fit the KMeans model to the data
kmeans.fit(pca_ds)

# Predict the cluster labels
cluster_labels = kmeans.predict(pca_ds)
# Assign cluster labels to the DataFrame
df['kmeans_clusters'] = cluster_labels
pca_ds["kmeans_clusters"] = cluster_labels
df1["kmeans_clusters"] = cluster_labels
silhouette = silhouette_score(pca_ds, cluster_labels)
davies_bouldin = davies_bouldin_score(pca_ds, cluster_labels)
calinski_harabasz = calinski_harabasz_score(pca_ds, cluster_labels)

# Store important scores in a dictionary
kmeans_scores = {
    'Silhouette Score': silhouette,
    'Davies-Bouldin Index': davies_bouldin,
    'Calinski-Harabasz Index': calinski_harabasz
}

print(kmeans_scores)




In [ ]:
pca_ds.head()


In [ ]:
# Set axis 
from sklearn.preprocessing import MinMaxScaler,StandardScaler

x = pca_ds["PC1"]
y = pca_ds["PC2"]
z = pca_ds["PC3"]



#Plotting the clusters
fig = plt.figure(figsize=(10,8))
ax = plt.subplot(111, projection='3d', label="bla")
ax.scatter(x, y, z, s=40, c=pca_ds["kmeans_clusters"], marker='o',cmap='viridis')
ax.set_title("The Plot Of The kmeans_clusters")
ax.legend()
plt.show()


In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
#  PCA transformation with cluster labels



In [ ]:
# Perform Hierarchical Clustering
from sklearn.cluster import AgglomerativeClustering

hierarchical_model = AgglomerativeClustering(n_clusters=4)
hierarchical_labels = hierarchical_model.fit_predict(pca_ds)
df['Hierarchical_Clusters'] = hierarchical_labels
pca_ds["Hierarchical_Clusters"] = hierarchical_labels
df1['Hierarchical_Clusters']=hierarchical_labels
silhouette = silhouette_score(pca_ds, cluster_labels)
davies_bouldin = davies_bouldin_score(pca_ds, cluster_labels)
calinski_harabasz = calinski_harabasz_score(pca_ds, cluster_labels)

# Store important scores in a dictionary
Hierarchical_Clusters_scores = {
    'Silhouette Score': silhouette,
    'Davies-Bouldin Index': davies_bouldin,
    'Calinski-Harabasz Index': calinski_harabasz
}

print(Hierarchical_Clusters_scores)

df.head()

In [ ]:
df1.head()

In [ ]:
# Set axis 
from sklearn.preprocessing import MinMaxScaler,StandardScaler

x = pca_ds["PC1"]
y = pca_ds["PC2"]
z = pca_ds["PC3"]



#Plotting the clusters
fig = plt.figure(figsize=(10,8))
ax = plt.subplot(111, projection='3d', label="bla")
ax.scatter(x, y, z, s=40, c=pca_ds["Hierarchical_Clusters"], marker='o',cmap='viridis')
ax.set_title("The Plot Of The Hierarchical_Clusters")
ax.legend()
plt.show()

In [ ]:
print(kmeans_scores)
print(Hierarchical_Clusters_scores)
# hierarchical_Clusters is better at clustering the points 
'''
    For Silhouette Score and Calinski-Harabasz Index, higher values are preferred.
    For Davies-Bouldin Index, lower values are preferred.
'''


In [ ]:

plt.figure(figsize=(13,8))
pl = sns.countplot(x=df['Hierarchical_Clusters'])
pl.set_title('Distribution Of The Clusters')
plt.show()

In [ ]:

plt.figure(figsize=(10,8))
df1['Spent'] = df1['MntWines'] + df1['MntFruits'] + df1['MntMeatProducts'] + df1['MntFishProducts'] + df1['MntSweetProducts'] + df1['MntGoldProds']
pl = sns.scatterplot(data=df1, x=df1['Spent'], y=df1['Income'], hue=df1['Hierarchical_Clusters'],cmap='coolwarm')
pl.set_title("Cluster's Profile Based on Income and Spending")
pl.set_xlabel('Total Spending')
pl.set_ylabel('Income')

# Adjust axis limits for better clarity
plt.xlim(0, 2500)  # Set x-axis limits
plt.ylim(0, 160000)  # Set y-axis limits
# Show legend
plt.legend()

plt.show()

# too much standardizaion have to see on non stadardized data 

In [ ]:
'''
Group 0: those with high  income and high spending 
Group 1: those with low  income and low spending 
Group 2: those with high income and moderate spending 
Group 3: those with moderate income and high spending
'''


In [ ]:
df['Total_Promos'] = df['AcceptedCmp1']+ df['AcceptedCmp2']+ df['AcceptedCmp3']+ df['AcceptedCmp4']+ df['AcceptedCmp5']
plt.figure(figsize=(10,8))
pl = sns.countplot(x=df['Total_Promos'], hue=df['Hierarchical_Clusters'])
pl.set_title('Count Of Promotion Accepted')
pl.set_xlabel('Number Of Total Accepted Promotions')
plt.legend(loc='upper right')
plt.show()

In [ ]:
'''
cluster 2 is mostly likely to accept promotions 
'''
''' promotion 1 is the mostly successfully promotion which appeals to clusters of customers'''

In [ ]:
df1.drop('Spent',axis=1)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Assuming you have already selected the clusters
selected_clusters= df1[df1['Hierarchical_Clusters'].isin([0])]


# Selecting only numerical columns for distribution plot
numerical_columns = selected_clusters.select_dtypes(include=['float64', 'int64'])

# Plotting distribution of numerical columns using displot with subplots
plt.figure(figsize=(15, 5*selected_clusters.shape[1]))
for i, column in enumerate(numerical_columns.columns):
    plt.subplot(numerical_columns.shape[1], 2, i+1)
    sns.histplot(selected_clusters[column], kde=True)
    plt.title(f'Distribution of {column} in Cluster 0')
    plt.xlabel('Value')
    plt.ylabel('Density')

plt.tight_layout()
plt.show()
selected_clusters.describe()


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Assuming you have already selected the clusters
selected_clusters= df1[df1['Hierarchical_Clusters'].isin([1])]


# Selecting only numerical columns for distribution plot
numerical_columns = selected_clusters.select_dtypes(include=['float64', 'int64'])

# Plotting distribution of numerical columns using displot with subplots
plt.figure(figsize=(15, 5*selected_clusters.shape[1]))
for i, column in enumerate(numerical_columns.columns):
    plt.subplot(numerical_columns.shape[1], 2, i+1)
    sns.histplot(selected_clusters[column], kde=True)
    plt.title(f'Distribution of {column} in Cluster 1')
    plt.xlabel('Value')
    plt.ylabel('Density')

plt.tight_layout()
plt.show()
selected_clusters.describe()


In [ ]:
''' in cluster0 
customers mostly likely to have  graduation degrees
income in the range of 31907-105471 with a mdeian income of 78423.500000	
they are unlikely to have kids or teenagers 
median age of 52 years and range of 29-81
'''


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Assuming you have already selected the clusters
selected_clusters = df1[df1['Hierarchical_Clusters'].isin([2])]

# Selecting only numerical columns for distribution plot
numerical_columns = selected_clusters.select_dtypes(include=['float64', 'int64'])

# Plotting distribution of numerical columns using displot with subplots
plt.figure(figsize=(15, 5*selected_clusters.shape[1]))
for i, column in enumerate(numerical_columns.columns):
    plt.subplot(numerical_columns.shape[1], 2, i+1)
    sns.histplot(selected_clusters[column], kde=True)
    plt.title(f'Distribution of {column} in Cluster 2')
    plt.xlabel('Value')
    plt.ylabel('Density')

plt.tight_layout()
plt.show()

selected_clusters.describe()

In [ ]:
'''
in cluster 1 
The majority of individuals fall  are likely to have graduation 
median income is 34081 with a range 1730-162397
they have mostly likely to have 1 kid 
median age of 51 and range of 28-84
mostly likely to be married 

'''

In [ ]:
'''
cluster2 
they are most likely to have graduation or a phd 
they have a high median income of 81,128
they have mostly likely to not have kids or teens 
they have a median age of 53 years of range of 29-81
'''

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Assuming you have already selected the clusters
selected_clusters = df1[df1['Hierarchical_Clusters'].isin([3])]

# Selecting only numerical columns for distribution plot
numerical_columns = selected_clusters.select_dtypes(include=['float64', 'int64'])

# Plotting distribution of numerical columns using displot with subplots
plt.figure(figsize=(15, 5*selected_clusters.shape[1]))
for i, column in enumerate(numerical_columns.columns):
    plt.subplot(numerical_columns.shape[1], 2, i+1)
    sns.histplot(selected_clusters[column], kde=True)
    plt.title(f'Distribution of {column} in Cluster 3')
    plt.xlabel('Value')
    plt.ylabel('Density')

plt.tight_layout()
plt.show()
selected_clusters.describe()

In [ ]:

import pickle
# Save the pipeline to a file
filename = 'customer_segment.pkl'
with open(filename, 'wb') as file:
    pickle.dump(pipeline, file)

# Load the model from disk
with open(filename, 'rb') as file:
    loaded_model = pickle.load(file)

print(loaded_model)

In [ ]:
'''
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import AgglomerativeClustering
from sklearn.metrics import silhouette_score, davies_bouldin_score, calinski_harabasz_score
import streamlit as st

# Streamlit setup
st.set_page_config(layout="wide")

# Define preprocessing steps
preprocessor = Pipeline([
    ('standardize', StandardScaler()),
    ('pca', PCA(n_components=3))
])

# Define the clustering model
clustering_model = AgglomerativeClustering()

# Create the pipeline
pipeline = Pipeline([
    ('preprocessing', preprocessor),
    ('clustering', clustering_model)
])

@st.cache
def load_data(file):
    try:
        data = pd.read_excel(file)
        return data
    except Exception as e:
        st.error(f"An error occurred while reading the file: {e}")
        return None

def main():
    st.title("Clustering Web App")

    uploaded_file = st.file_uploader("Upload Excel file", type=["xlsx"])

    if uploaded_file is not None:
        data = load_data(uploaded_file)
        if data is not None:
            cluster_labels = pipeline.fit_predict(data)
            results = {
                'Silhouette Score': silhouette_score(data, cluster_labels),
                'Davies-Bouldin Index': davies_bouldin_score(data, cluster_labels),
                'Calinski-Harabasz Index': calinski_harabasz_score(data, cluster_labels),
                'Cluster Labels': cluster_labels.tolist()
            }

            st.write("Clustering Results:")
            st.write(results)

if __name__ == "__main__":
    main()
'''

In [1]:
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA,TruncatedSVD
from sklearn.cluster import AgglomerativeClustering
from sklearn.metrics import silhouette_score, davies_bouldin_score, calinski_harabasz_score
import streamlit as st
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder


# Streamlit setup
st.set_page_config(layout="wide")

# Define preprocessing steps
preprocessor = Pipeline([
    ('encoding', OneHotEncoder()),
    ('standardize', StandardScaler(with_mean=False)),
    ('svd', TruncatedSVD(n_components=3))  # Use TruncatedSVD instead of PCA
])

# Define the clustering model
clustering_model = AgglomerativeClustering()

# Create the pipeline
pipeline = Pipeline([
    ('preprocessing', preprocessor),
    ('clustering', clustering_model)
])

def load_data(file):
    try:
        data = pd.read_excel(file)
        return data
    except Exception as e:
        st.error(f"An error occurred while reading the file: {e}")
        return None

def main():
    st.title("Clustering Web App")

    uploaded_file = st.file_uploader("Upload Excel file", type=["xlsx"])

    if uploaded_file is not None:
        data = load_data(uploaded_file)
        if data is not None:
            cluster_labels = pipeline.fit_predict(data)
            results = {
                'Silhouette Score': silhouette_score(data, cluster_labels),
                'Davies-Bouldin Index': davies_bouldin_score(data, cluster_labels),
                'Calinski-Harabasz Index': calinski_harabasz_score(data, cluster_labels),
                'Cluster Labels': cluster_labels.tolist()
            }

            st.write("Clustering Results:")
            st.write(results)

# Create a POST API
from flask import Flask, request, jsonify

app = Flask(__name__)

@app.route('/predict', methods=['POST'])
def predict():
    if 'file' not in request.files:
        return jsonify({'error': 'No file uploaded'}), 400
    
    file = request.files['file']
    
    if file.filename == '':
        return jsonify({'error': 'Empty file uploaded'}), 400
    
    if not file.filename.endswith('.xlsx'):
        return jsonify({'error': 'Invalid file type. Only Excel files (.xlsx) are allowed.'}), 400
    
    data = load_data(file)
    if data is None:
        return jsonify({'error': 'An error occurred while reading the file.'}), 400
    
    cluster_labels = pipeline.fit_predict(data)
    results = {
        'Silhouette Score': silhouette_score(data, cluster_labels),
        'Davies-Bouldin Index': davies_bouldin_score(data, cluster_labels),
        'Calinski-Harabasz Index': calinski_harabasz_score(data, cluster_labels),
        'Cluster Labels': cluster_labels.tolist()
    }

    return jsonify(results), 200

if __name__ == "__main__":
    main()
    app.run(port=8000)


2024-03-13 10:14:14.357 
  command:

    streamlit run C:\Users\Riddhima\AppData\Roaming\Python\Python38\site-packages\ipykernel_launcher.py [ARGUMENTS]


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:8000
Press CTRL+C to quit
127.0.0.1 - - [13/Mar/2024 10:14:41] "GET / HTTP/1.1" 404 -
127.0.0.1 - - [13/Mar/2024 10:14:41] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [13/Mar/2024 10:14:50] "GET /predict HTTP/1.1" 405 -
